In this final solution, we will be incorporating neural networks to see if we can boost the modeling performance. As the previous solution, we will first start by importing the necessary modules and will then load the dataset. 

In [1]:
# Filter the uneccesary warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import numpy
import numpy as np

# Fix the random seed
np.random.seed(7)

In [3]:
# Load the numpy arrays which will be our datasets from now
X_train, y_train = np.load("X_train.npy", allow_pickle=True), np.load("y_train.npy", allow_pickle=True)
X_test, y_test = np.load("X_test.npy", allow_pickle=True), np.load("y_test.npy", allow_pickle=True)

In [4]:
# Other imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import *
from wandb.keras import WandbCallback
import wandb

In [5]:
# Fix TensorFlow's random see for better reproducibility
import tensorflow as tf
tf.random.set_seed(666)

I find it convenient to wrap my model definition inside a function and return a compiled version of the model from the function. 

In [6]:
# Model building using the Sequential API
def get_training_model(data):
    model = Sequential()

    model.add(Dense(40, activation="relu",
              kernel_initializer="uniform",input_dim=data.shape[1]))
    model.add(Dense(30, activation="relu",
              kernel_initializer="uniform"))
    model.add(Dense(1, activation="sigmoid"))

    model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])
    return model

When using the `ReLU` activation function it's good to use the `He_Uniform` scheme to initialize the weights of your neural network and that can be set using the `kernel_initializer` argument. 

In [7]:
# Instantiate the model and print its summary
model = get_training_model(X_train)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40)                1240      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1230      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 2,501
Trainable params: 2,501
Non-trainable params: 0
_________________________________________________________________


Early Stopping is a good way to prevent overfitting and in many cases you won't need to train the model for all the epochs. If you see that you model's performance is not as expected the training will get stopped after fixed interval (which will be set by you) and the best model within those many epochs will be returned. Let's define an `EarlyStopping` callback. 

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
es_cb = EarlyStopping(monitor="loss", 
    patience=5, # number of epochs to consider
    restore_best_weights=True, # restore the best weights when loss stops enhancing
    verbose=1)

Let's start training our shallow network. 

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import time 

In [11]:
def train_network(model, name):
    # Initialize Weights and Biases
    wandb.init(project="phishing-websites-detection", name=name)
    
    start = time.time()
    history = model.fit(X_train, y_train, batch_size=64, epochs=128, verbose=1, \
        callbacks=[es_cb, WandbCallback()])
    end = time.time()-start
    prediction = model.predict_classes(X_test)
    wandb.log({"accuracy":accuracy_score(y_test, prediction)*100.0,\
               "precision": precision_recall_fscore_support(y_test, prediction, average="macro")[0],
               "recall": precision_recall_fscore_support(y_test, prediction, average="macro")[1],
               "training_time":end})
    print("Accuracy score of the Logistic Regression classifier with default hyperparameter values {0:.2f}%"\
                  .format(accuracy_score(y_test, prediction)*100.))
    print("\n")
    print("----Classification report of the Logistic Regression classifier with default hyperparameter value----")
    print("\n")
    print(classification_report(y_test, prediction, target_names=["Phishing Websites", "Normal Websites"]))

I would encourage you to specify other things like the data, number of epochs, batch size as arguments to the function to have more flexibility and control. 

In [12]:
train_network(model, "neural-network")

wandb: Wandb version 0.8.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Train on 8844 samples
Epoch 1/128
8844/8844 [==============================] - 1s 109us/sample - loss: 0.3573 - accuracy: 0.8673
Epoch 2/128
8844/8844 [==============================] - 0s 28us/sample - loss: 0.1969 - accuracy: 0.9237
Epoch 3/128
8844/8844 [==============================] - 0s 26us/sample - loss: 0.1883 - accuracy: 0.9249
Epoch 4/128
8844/8844 [==============================] - 0s 22us/sample - loss: 0.1785 - accuracy: 0.9292
Epoch 5/128
8844/8844 [==============================] - 0s 27us/sample - loss: 0.1752 - accuracy: 0.9294
Epoch 6/128
8844/8844 [==============================] - 0s 26us/sample - loss: 0.1683 - accuracy: 0.9341
Epoch 7/128
8844/8844 [==============================] - 0s 26us/sample - loss: 0.1607 - accuracy: 0.9358
Epoch 8/128
8844/8844 [==============================] - 0s 23us/sample - loss: 0.1546 - accuracy: 0.9383
Epoch 9/128
8844/8844 [==============================] - 0s 25us/sample - loss: 0.1503 - accuracy: 0.9391
Epoch 10/128
8844/8844 

We already have a training accuracy of 97%. Can we push this even further? 

**TDLHBA** is technique introduced [in this paper](https://dl.acm.org/citation.cfm?id=3227655). We will use hyperparameter values as presented in the paper to see the performance enhancement of the model.

In [13]:
# Building the model with the same topology as specified in the above-mentioned paper

model_TDLHBA = Sequential()

model_TDLHBA.add(Dense(40, activation="relu",
          kernel_initializer="uniform", input_dim=X_train.shape[1]))
model_TDLHBA.add(Dense(30, activation="relu",
          kernel_initializer="uniform"))
model_TDLHBA.add(Dense(1, activation="sigmoid"))

adam = Adam(lr=0.0017470)
model_TDLHBA.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])

In [14]:
wandb.init(project="phishing-websites-detection", name="neural-network-tdlhba")

wandb: Wandb version 0.8.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


W&B Run: https://app.wandb.ai/datagodzilla/phishing-websites-detection/runs/rvoco64x

In [15]:
start = time.time()
history_TDLHBA = model_TDLHBA.fit(X_train, y_train, batch_size=10, epochs=100, 
                                  verbose=1, callbacks=[es_cb, WandbCallback()])

end = time.time() - start
prediction = model_TDLHBA.predict_classes(X_test)
wandb.log({"accuracy":accuracy_score(y_test, prediction)*100.0,\
           "precision": precision_recall_fscore_support(y_test, prediction, average="macro")[0],
           "recall": precision_recall_fscore_support(y_test, prediction, average="macro")[1],
           "training_time":end})
print("Accuracy score of the Logistic Regression classifier with default hyperparameter values {0:.2f}%"\
                  .format(accuracy_score(y_test, prediction)*100.))
print("\n")
print("----Classification report of the Logistic Regression classifier with default hyperparameter value----")
print("\n")
print(classification_report(y_test, prediction, target_names=["Phishing Websites", "Normal Websites"]))

Train on 8844 samples
Epoch 1/100


wandb: Wandb version 0.8.28 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


8844/8844 [==============================] - 5s 544us/sample - loss: 0.2218 - accuracy: 0.9088
Epoch 2/100
8844/8844 [==============================] - 2s 261us/sample - loss: 0.1705 - accuracy: 0.9305
Epoch 3/100
8844/8844 [==============================] - 2s 269us/sample - loss: 0.1526 - accuracy: 0.9365
Epoch 4/100
8844/8844 [==============================] - 3s 306us/sample - loss: 0.1358 - accuracy: 0.9430
Epoch 5/100
8844/8844 [==============================] - 2s 208us/sample - loss: 0.1243 - accuracy: 0.9490
Epoch 6/100
8844/8844 [==============================] - 2s 199us/sample - loss: 0.1158 - accuracy: 0.9498
Epoch 7/100
8844/8844 [==============================] - 2s 273us/sample - loss: 0.1088 - accuracy: 0.9535
Epoch 8/100
8844/8844 [==============================] - 3s 306us/sample - loss: 0.1010 - accuracy: 0.9569
Epoch 9/100
8844/8844 [==============================] - 3s 286us/sample - loss: 0.0964 - accuracy: 0.9595
Epoch 10/100
8844/8844 [=========================

The performance is by far the best one. 